# Reconocimiento de patrones: Identificación de grupos o Clustering


# Distancias  y Grupos

__Clustering__ se refiere a un conjunto muy amplio de técnicas para encontrar subgrupos, o grupos, en un conjunto de datos. Cuando agrupamos las observaciones de un conjunto de datos, buscamos dividirlas en grupos distintos para que las observaciones dentro de cada grupo sean bastante similares entre sí, al mismo tiempo que las observaciones en los diferentes grupos sean lo suficientemente diferentes entre ellas.

Por supuesto, para hacer que esto sea concreto, debemos definir lo que significa que dos o más observaciones sean similares o diferentes. De hecho, esta es a menudo una consideración específica del dominio que debe realizarse en función del conocimiento de los datos que se estudian. Por ejemplo, supongamos que tenemos un conjunto de $n$ observaciones, cada una con $p$ características. Las $n$ observaciones podrían corresponder a muestras de tejido para pacientes con cáncer de mama, y las $p$ características corresponden a mediciones recogidas para cada muestra de tejido; estas podrían ser medidas clínicas, como la etapa o grado del tumor, o podrían ser mediciones de la expresión génica. Podemos tener una razón para creer que existe alguna heterogeneidad entre las n muestras de tejido; por ejemplo, tal vez haya algunos diferentes subtipos desconocidos de cáncer de mama. La agrupación podría usarse para encontrar estos subgrupos. Este es un problema no supervisado porque estamos tratando de descubrir la estructura (en este caso, clusters distintos) sobre la base de un conjunto de datos.


## Definición

El reconocimiento de patrones se realiza en un ciclo de tres fases principales:

1. La identificación o definición de clases en las que se pueden agrupar los elementos de un dominio de interés. Estas clases suelen ser creadas a partir de la observación de características que presentan los elementos bajo observación.
2. La asignación de elementos (conocidos) a clases definidas.
3. La predicción de nuevas observaciones dada la dinámica observada a partir del conjunto de clases.


Para realizar la construcción de las clases a partir de observaciones, utilizamos un técnica conocida como  *identificación de grupos*, o más comunmente, "*clustering*".

El clustering consiste en agrupar objetos en grupos de tal manera que los objetos pertenecientes a un grupo (o "*cluster*") son más semejantes entre sí que a otros objetos no pertenecientes al grupo.

![](images/clusters_aficionados.jpg)
![ ](images/blank.png)

La semejanza se define a través de diversos rasgos, algunos de los cuales pueden ser de carácter semántico; En la imagen a continuación, por ejemplo, pueden distinguirse dos grupos principales de fans: los "*amarillos*" que junto con los "*azules*" forman el grupo de "*fans brasileños*" en oposición al grupo de fans "*verdes*" y fans "*rojos*" que conforman el grupo de "*fans mexicanos*".

El agrupamiento es una habilidad natural de los seres vivos. Las relaciones de semejanza nos permiten identificar grupos incluso como si fueran objetos inexistentes:

![](images/law-of-proximity.png)
![ ](images/blank.png)
![](images/Pattern1.jpg)
![ ](images/blank.png)
![ ](images/blank.png)
![](images/gestalt_proximity_dalmation_by_gderanidaye.png)



## Técnicas de clustering: *K-Means*

La técnica de *k-medias* (o *k-means*) es una de las técnicas de clustering más simples y más utilizadas. Es una técnica de *aprendizaje no supervisado*: la técnica debe clasificar objetos desconocidos a un conjunto de clases de las que apenas se conocen algunos parámetros y no las características de cada clase. En el caso de *k*-medias, lo único que se conoce es el número de grupos.

El objetivo del algoritmo de *k*-medias es particionar un conjunto de *n* datos en *k* grupos, asociando objetos cercanos y distinguiendo objetos diferentes. El resultado al final del procedimiento es la generación de un "*prototipo de clase*".

En la imagen a continuación, se presentan 30 datos (puntos azules) que han sido agrupados en 4 clusters. Las líneas rojas representan fronteras (posibles) entre clase y las estrellas rojas representan prototipos de clase.

![ ](images/k-means0.png)

La clase es una generalización del cluster; una descripción conceptual del grupo representado por el conjunto de datos. La delimitación de una clase, construida de forma inductiva (a partir de ejemplos), suele no ser precisa: está limitada a los datos de que disponemos para modelarla, como se ilustra en la imagen siguiente. 

![ ](images/cluster-class.png)

En esta imagen, los datos (ficticios) conocidos nos han permitido generar un modelo de clase (area en azul). Sin embargo, la clase real es diferente, sólo que al crear la clase no conocíamos suficientes datos para modelarla fielmente (como los datos señalados como puntos rojos). Considérese, por ejemplo, el siguiente conjunto de bicicletas:

![ ](images/bicycles.png)

A partir de los datos disponibles podemos concluir, por ejemplo, que todas las bicicletas tienen un asiento y pedales... sin embargo, esta definción de la clase bicicleta no incluye los siguientes ejemplares:

[![ ](images/bicycles-2.png)](https://www.youtube.com/watch?v=KewMZ8sM0Uo)


### Algoritmo

El algoritmo k-means sigue los siguientes pasos (dado un conjunto de datos):

In [41]:
# Inicializar el ambiente
import numpy as np
import pandas as pd
import math
import random
import time
import os
import sys
from scipy.spatial import distance
np.set_printoptions(precision=2, suppress=True) # Cortar la impresión de decimales a 1

LARGER_DISTANCE = sys.maxsize
TALK = True # TALK = True, imprime resultados parciales

In [42]:
# Leer los datos de archivo
DATA_SET = pd.read_csv("data/datosProm.csv", names = ['A', 'B']).values
DATA_LEN = len(DATA_SET)

# Definir una clase para expresar puntos y su asignación a un cluster
# éste código no es necesario que sea comprendido, es para ilustrar los ejemplos
class DataPoint:
    def __init__(self, p):
        self.value = p[:]
        
    def set_value(self, p):
        self.value = p
    
    def get_value(self):
        return self.value
    
    def set_cluster(self, cluster):
        self.cluster = cluster
    
    def get_cluster(self):
        return self.cluster

data = []
def initialize_dataset():
    for i in range(DATA_LEN):
        point = DataPoint(DATA_SET[i])
        point.set_cluster(None)
        data.append(point)
    return

# --------------------------
# Crear el conjunto de datos
initialize_dataset()

1) Definir el valor de $k$

In [43]:
NUM_CLUSTERS = 3

2) Seleccionar de manera arbitraria *k* puntos en el espacio de características como centros iniciales de  los clusters (centroides o centros de masa).
![ ](images/k-means1.png)

In [44]:
# Definir forma de muestreo; 0 = random, 1=head, 2=tail
SAMPLING_METHOD = 1

centroids = []
def initialize_centroids():
    if (TALK) : 
        print("Centroides inicializados en:")
    for c in range(NUM_CLUSTERS):
        if (SAMPLING_METHOD == 0) :
            which = random.randint(0,DATA_LEN-1)
        elif (SAMPLING_METHOD == 1):
            which = c
        else :
            which = DATA_LEN-1 - c
                
        centroids.append(list(DATA_SET[which]))
        if (TALK) : 
            print(centroids[c])        
    if (TALK) : 
        print()
    
    return

# --------------------------
# Inicializar los centroides
initialize_centroids()

Centroides inicializados en:
[70.28, 42.125]
[0.0, 56.75]
[79.0, 2.5]



3) Asignar cada punto del conjunto de datos al cluster donde la distancia del punto al centroide es menor.
![ ](images/k-means2.png)

In [45]:
def update_clusters():
    changed = False
    
    for i in range(DATA_LEN):
        minDistance = LARGER_DISTANCE
        currentCluster = 0
        
        for j in range(NUM_CLUSTERS):
            dist = distance.euclidean(data[i].get_value(), centroids[j])
            if(dist < minDistance):
                minDistance = dist
                currentCluster = j
        
        if(data[i].get_cluster() is None or data[i].get_cluster() != currentCluster):
            data[i].set_cluster(currentCluster)
            changed = True
            
    members = [0] * NUM_CLUSTERS
    for i in range(DATA_LEN):
        members[data[i].get_cluster()] += 1
    
    if (TALK) : 
        for j in range(NUM_CLUSTERS):
            print("El cluster ", j+1, " incluye ", members[j], "miembros.")
        print()
            
    return changed

# --------------------------
# Actualizar los clusters
KEEP_WALKING = update_clusters()

El cluster  1  incluye  18 miembros.
El cluster  2  incluye  7 miembros.
El cluster  3  incluye  5 miembros.



4) Calcular los centroides a partir de los puntos en cada cluster. 
![ ](images/k-means3.png)

In [46]:
def update_centroids():    
    if (TALK) : 
        print("Los nuevos centroids son:")
    for j in range(NUM_CLUSTERS):
        means = [0] * DATA_SET.shape[1]
            
        clusterSize = 0
        for k in range(len(data)):
            if(data[k].get_cluster() == j):
                p = data[k].get_value()
                for i in range(DATA_SET.shape[1]):
                    means[i] += p[i]
                clusterSize += 1

        if(clusterSize > 0):
            for i in range(DATA_SET.shape[1]):
                centroids[j][i] = means[i] / clusterSize

        if (TALK) : 
            print(centroids[j])        
    if (TALK) : 
        print()
    
    return

# --------------------------
# Actualizar los centroides
update_centroids()

Los nuevos centroids son:
[80.95777777777778, 73.79605555555554]
[0.0, 73.88657142857141]
[84.00999999999999, 6.466800000000001]



5) Repetir los pasos 2 y 3 hasta que no haya cambios en los clusters.
![ ](images/k-means4.png)

In [47]:
while(KEEP_WALKING):
    KEEP_WALKING = update_clusters()
    if (KEEP_WALKING):
        update_centroids()
    else :
        if (TALK) : 
            print ("No más cambios.") 

El cluster  1  incluye  17 miembros.
El cluster  2  incluye  7 miembros.
El cluster  3  incluye  6 miembros.

Los nuevos centroids son:
[81.55529411764707, 76.07817647058822]
[0.0, 73.88657142857141]
[81.80833333333332, 11.222333333333333]

El cluster  1  incluye  16 miembros.
El cluster  2  incluye  7 miembros.
El cluster  3  incluye  7 miembros.

Los nuevos centroids son:
[82.25999999999999, 78.20024999999998]
[0.0, 73.88657142857141]
[80.16142857142857, 15.637]

El cluster  1  incluye  16 miembros.
El cluster  2  incluye  7 miembros.
El cluster  3  incluye  7 miembros.

No más cambios.


### Ejemplos lógicos

#### Estrategias de segmentación de mercado

<img src="images/k-means_1.png" alt="Drawing" style="width:900px;"/>

si analizamos los datos podemos dividir los clientes de la siguiente manera:

<img src="images/k_means_3.png" alt="Drawing" style="width:900px;"/>

Un ejemplo clásico de k-means es el problema de las pizerías:
    - Partimos de un estudio de mercado
    - ¿Cuantas pizerías debemos crear?

<img src="images/k_means_4.png" alt="Drawing" style="width:900px;"/>

segun el ojo humano serían 3, pero ¿cómo lo planteamos en un algoritmo?

<img src="images/k_means_5.png" alt="Drawing" style="width:900px;"/>

Creamos sucursales al azar, dentro del rango geográfico

<img src="images/k_means_6.png" alt="Drawing" style="width:900px;"/>

dos pasos esenciales ...

<img src="images/k_means_7.png" alt="Drawing" style="width:900px;"/>

reubicar sucursales de acuerdo a estrategía de mercado

<img src="images/k_means_8.png" alt="Drawing" style="width:900px;"/>

repetir asignación de clientes de acuerdo a la sucursal más cercana

<img src="images/k_means_9.png" alt="Drawing" style="width:900px;"/>

entonces ...

<img src="images/k_means_10.png" alt="Drawing" style="width:900px;"/>

reasignar centroides  y repetir ...

<img src="images/k_means_11.png" alt="Drawing" style="width:900px;"/>

¿fin ?

<img src="images/k_means_12.png" alt="Drawing" style="width:900px;"/>

<div class="alert alert-info" role="alert"><i class="fa fa-lightbulb-o" aria-hidden="true"></i> <strong> Ejemplo de clustering </strong> https://www.youtube.com/watch?v=BVFG7fd1H30</div>


<hr style="border-width: 1px;">

El método $k$-means es un método muy robusto (aunque diversos autores han buscado maneras de eficientarlo, ver por ejemplo [este artículo](http://worldcomp-proceedings.com/proc/p2015/CSC2667.pdf) o [este otro](http://academics.smcvt.edu/jtrono/Papers/SMCClusteringPaper_DavidKronenberg.pdf). Su principal restricción es la __selección adecuada del número de clusters, $k$__ . En casos más interesantes que nuestro ejemplo, conviene realizar un análisis previo de determinación de clusters, por dendrogramas.




<div class="alert alert-info" role="alert"><i class="fa fa-lightbulb-o" aria-hidden="true"></i> <strong> Más  sobre Clusterig</strong> http://scikit-learn.org/stable/modules/clustering.html#k-means</div>

El algoritmo k-means es extremadamente fácil de implementar pero también es computacionalmente muy eficiente en comparación con otros algoritmos de clustering, lo que podría explicar su popularidad. El algoritmo $k$-means pertenece a la categoría de __clustering basado en prototipos__ . Existen otras dos categorías de clustering muy populares, __jerarquización__ y __agrupamiento basado en densidad__ . El agrupamiento basado en prototipos significa que cada grupo está representado por un prototipo, que puede ser el centroide (promedio) de puntos similares con características continuas, o el medoide (el punto más representativo o más frecuente) en el caso de las características categóricas. 

Mientras que $k$-means es muy bueno para identificar grupos de formas esféricas, uno de los inconvenientes de este algoritmo de agrupación es que tenemos que especificar el número de clústers $k$ a priori. Una elección inadecuada para $k$ puede dar como resultado un rendimiento de clúster pobre. Más adelante en este capítulo, discutiremos el método del codo y los diagramas de silueta, que son técnicas útiles para evaluar la calidad de una agrupación para ayudarnos a determinar la cantidad óptima de conglomerados $k$.


Existen variantes de  $k$- Means (tambien llamado $C$-Means) que toman en cuenta  otros tipos de distancias o punto de los vectores protitipos (media, moda, etc.). Dos variantes populares son : k-medoids y fuzzy C-means

### K-means ++
Hasta ahora, hemos discutido el algoritmo clásico _k-means_ que usa una semilla aleatoria para ubicar los centroides iniciales, lo que a veces puede resultar en agrupamientos incorrectos o convergencia lenta si los centroides iniciales se eligen mal. Una forma de abordar este problema es ejecutar el algoritmo k-means varias veces en un conjunto de datos y elegir el modelo de mejor rendimiento. 

Otra estrategia es __ubicar los centroides iniciales lejos el uno del otro a través del algoritmo _k-means_ ++__ , lo que conduce a mejores resultados y más consistentes que los medios k clásicos (D. Arthur y S. Vassilvitskii. K-means ++. Society for Industrial and Applied Mathematics, 2007).

Otro problema con _k-means_ es que uno o más clusters pueden estar vacíos. Tenga en cuenta que este problema no existe para k-medoids o Fuzzy C-means, un algoritmo que discutiremos en la siguiente subsección. Sin embargo, este problema se tiene en cuenta en la implementación actual de k-medias en scikit-learn. Si un clúster está vacío, el algoritmo buscará la muestra más alejada del centro de gravedad del clúster vacío. Luego reasignará el centroide para que sea el punto más lejano


### Hard vs soft clustering
La agrupación en clúster duro describe una familia de algoritmos donde cada muestra de un conjunto de datos se asigna exactamente a un clúster, como en el algoritmo de k-medias que discutimos en la subsección anterior. Por el contrario, los algoritmos para clústeres blandos (a veces también denominados agrupamientos difusos) asignan una muestra a uno o más clústeres. Un ejemplo popular de clustering suave es el algoritmo C-means difuso (FCM) (también llamado soft k-means o fuzzy k-means). La idea original se remonta a la década de 1970, cuando Joseph C. Dunn propuso por primera vez una versión temprana de clustering difuso para mejorar k-means (_J. C. Dunn.
A Fuzzy Relative of the Isodata Process and its Use in Detecting Compact Well-separated Clusters. 1973_). Casi una década después, James C. Bedzek publicó su trabajo sobre las mejoras del algoritmo de agrupamiento difuso, que ahora se conoce como el algoritmo FCM (_J. C. Bezdek. Pattern Recognition with Fuzzy Objective Function Algorithms.
Springer Science & Business Media, 2013_).

### X-Means

Este algoritmo es una variante mejorada del K-Means. Su ventaja fundamental está en haber solucionado una de las mayores deficiencias presentadas en K-Means, el hecho de tener que __seleccionar a priori el número de clusters que se deseen obtener__ , a X-Means se le define un límite inferior K-min (número mínimo de clusters) y un límite superior K-Max (número máximo de clusters) y este algoritmo es capaz de obtener en ese rango el número óptimo de clusters, dando de esta manera más flexibilidad al usuario. Durante este proceso, el conjunto de centroides que alcanzan el mejor valor son almacenados, y estos serían la salida final, es decir, los valores finales de cada simulación de acuerdo a la distancia entre ellos. Los mismos son aplicables cuando en la Base de datos existen al menos 2 simulaciones para el modelo (que son ecuaciones formadas por arreglos de parámetros y condiciones iniciales). Se ha comprobado que sus resultados son más fiables que los obtenidos con el K-Means, debido a que presenta un valor de distorsión menor, son mucho mejor para realizar Clusters de un conjunto grande de datos y es incluso una variante mucho más rápida.



### Otros enfoques


#### Cobweb

Pertenece a la familia de algoritmos jerárquicos. Se caracteriza por la utilización de aprendizaje incremental, esto quiere decir, que realiza las agrupaciones instancia a instancia. Durante la ejecución del algoritmo se forma un árbol (árbol de clasificación) donde las hojas representan los segmentos y el nodo raíz engloba por completo el conjunto de datos. Al principio, el árbol consiste en un único nodo raíz. Las instancias se van añadiendo una a una y el árbol se va actualizando en cada paso. La clave para saber cómo y dónde se debe actualizar el árbol la proporciona una medida denominada utilidad de categoría, que mide la calidad general de una partición de instancias en un segmento. Pertenece a los métodos de aprendizaje conceptual o basado en modelos. Esto significa que cada cluster se considera como un modelo que puede describirse intrínsecamente, más que un ente formado por una colección de puntos. Además en el algoritmo también hay que tener en cuenta dos parámetros muy importantes:

Acuity: es un parámetro muy necesario, pues la utilidad de categoría está basada en la estimación de la media y la desviación estándar del valor de un atributo para un nodo en particular, el resultado es 0 si dicho nodo solo tiene una instancia; por lo que se puede decir que el valor que toma este parámetro es la medida del error de un nodo con una sola instancia (establece la varianza mínima de un atributo).

Cut-off: este parámetro es usado para evitar el crecimiento descontrolado de la cantidad de segmentos. Indica el grado de mejor ía que se debe producir en la utilidad de categoría para que la instancia se pueda tener en cuenta de manera individual. Resumiendo, cuando se va a añadir un nuevo nodo y no es suficiente el crecimiento de la utilidad de categoría, pues ese nodo se poda y la instancia pasa a otro nodo ya existente.

#### EM

Este algoritmo pertenece a una familia de modelos que se conocen como Finite Mixture Models, los cuales se pueden utilizar para segmentar conjuntos de datos. Está clasificado como un método de particionado y recolocación, o sea, Clustering Probabilístico. Se trata de obtener la FDP (Función de Densidad de Probabilidad) desconocida a la que pertenecen el conjunto completo de datos. El algoritmo EM, procede en dos pasos que se repiten de forma iterativa:

Expectation: Utiliza los valores de los parámetros, iniciales o proporcionados por el paso Maximization, obteniendo diferentes formas de la FDP buscada.

Maximization: Obtiene nuevos valores de los parámetros a partir de los datos proporcionados por el paso anterior.

Finalmente se obtendrá un conjunto de clusters que agrupan el conjunto de proyectos original. Cada uno de estos cluster estará definido por los parámetros de una distribución 